# 📊 Survey Correlation Analysis Notebook
This notebook:
- Cleans messy survey question columns (e.g., 'High-10', 'Low-1')
- Automatically extracts numeric values
- Calculates Pearson correlations and p-values
- Visualizes the correlation matrix
- Generates plain-English executive summaries

**💡 Designed to be flexible for any numeric-question-based survey dataset**

## 🔁 Step 1: Load Your Survey Data
Replace the file path with your dataset. Supports Excel or CSV formats.

In [ ]:
import pandas as pd
import numpy as np
import re
from scipy.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt

# Load your data (replace the filename with your actual file)
df = pd.read_excel("your_survey_file.xlsx")  # or pd.read_csv("your_file.csv")
df.head()

## 📋 Step 2: View and Select Columns to Clean
This prints the list of columns so you can identify your survey questions.

In [ ]:
print("📋 Column Names:")
print(df.columns.tolist())

## ✅ Step 3: Choose Which Columns to Clean
Select columns containing 1–10 scale responses (with or without text).

In [ ]:
# 🧠 OPTION A: Manually specify the columns you want to clean
# These should be columns that contain survey question responses using a numeric scale (e.g., 1–10)
# This is useful if your columns do not follow a consistent naming pattern like 'Q1', 'Q2', etc.
# 👉 Simply type your column names in the list below, using the exact names from df.columns

columns_to_clean = [
    # Example: 'Q1', 'Q2', 'Satisfaction_Score', 'Ease_of_Use'
    # Replace with your actual column names below
]

print("🧼 Manually selected columns for cleaning:")
print(columns_to_clean)

## 🧼 Step 4: Clean Textual Ratings Like 'High-10'
This function extracts the numeric part of strings such as 'Low-1', 'High-9', etc.

In [ ]:
def extract_numeric(val):
    if pd.isna(val):
        return np.nan
    if isinstance(val, str):
        match = re.search(r'\d+(\.\d+)?', val)
        return float(match.group()) if match else np.nan
    elif isinstance(val, (int, float)):
        return val
    else:
        return np.nan

In [ ]:
def extract_numeric(val):
    # If the value is missing (NaN), return it as-is
    if pd.isna(val):
        return np.nan

    # If the value is a string (e.g., 'High-10'), extract the numeric portion using regex
    if isinstance(val, str):
        match = re.search(r'\d+(\.\d+)?', val)  # This finds integer or decimal numbers
        return float(match.group()) if match else np.nan  # Return the number if found

    # If the value is already a number (int or float), return it as-is
    elif isinstance(val, (int, float)):
        return val

    # For all other types (e.g., lists, dicts, booleans), return NaN
    else:
        return np.nan

In [ ]:
for col in columns_to_clean:
    df[col] = df[col].apply(extract_numeric)

## ✅ Step 6: Optional - Keep Only Values Between 1 and 10

In [ ]:
for col in columns_to_clean:
    df[col] = df[col].apply(lambda x: x if pd.notna(x) and 1 <= x <= 10 else np.nan)

## 📈 Step 7: Calculate Correlation and P-values

In [ ]:
correlation_matrix = pd.DataFrame(index=columns_to_clean, columns=columns_to_clean, dtype=float)
p_value_matrix = pd.DataFrame(index=columns_to_clean, columns=columns_to_clean, dtype=float)

for col1 in columns_to_clean:
    for col2 in columns_to_clean:
        if col1 == col2:
            correlation_matrix.loc[col1, col2] = 1.0
            p_value_matrix.loc[col1, col2] = 0.0
        else:
            x = df[col1]
            y = df[col2]
            valid = x.notna() & y.notna()
            if valid.sum() > 2:
                r, p = pearsonr(x[valid], y[valid])
                correlation_matrix.loc[col1, col2] = r
                p_value_matrix.loc[col1, col2] = p
            else:
                correlation_matrix.loc[col1, col2] = np.nan
                p_value_matrix.loc[col1, col2] = np.nan

## 📊 Step 8: Visualize Correlation Matrix

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix.astype(float), annot=True, cmap='coolwarm', center=0)
plt.title("Correlation Matrix of Survey Questions")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig("correlation_heatmap.png")
plt.show()

## 📢 Step 9: Generate Executive Summary of Significant Correlations

In [ ]:
summary = ""
thresholds = [(0.7, "strong"), (0.4, "moderate"), (0.2, "weak")]

for col in correlation_matrix.columns:
    sentences = []
    for other_col in correlation_matrix.columns:
        if col != other_col:
            corr = correlation_matrix.loc[col, other_col]
            p_val = p_value_matrix.loc[col, other_col]
            if pd.notna(corr) and abs(corr) >= 0.2 and p_val < 0.05:
                strength = next((label for thresh, label in thresholds if abs(corr) >= thresh), "very weak")
                direction = "positive" if corr > 0 else "negative"
                sentence = f"- {col} is {strength}ly and {direction}ly correlated with {other_col} (r = {corr:.2f}, p = {p_val:.3f})"
                sentences.append(sentence)
    if sentences:
        summary += f"\n### {col}:\n" + "\n".join(sentences) + "\n"

# Save summary to file
with open("correlation_summary.txt", "w") as f:
    f.write(summary)
print("✅ Executive summary saved as: correlation_summary.txt")

## 💾 Step 10: Save Cleaned Data

In [ ]:
df.to_excel("cleaned_survey_file.xlsx", index=False)
print("✅ Cleaned data saved to: cleaned_survey_file.xlsx")